# HL Creep: Delayed Yielding and Viscosity Bifurcation

## Learning Objectives

1. Fit the Hébraud-Lequeux (HL) model to real polymer creep compliance data
2. Understand delayed yielding near the yield stress
3. Explore viscosity bifurcation behavior
4. Compare creep response above and below yield stress
5. Identify parameter identifiability challenges in creep fitting

## Prerequisites

- HL flow curve tutorial (hl/01_hl_flow_curve.ipynb)
- Bayesian inference fundamentals (bayesian/01-bayesian-basics.ipynb)

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~2-3 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~8-12 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import json
import os
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.hl import HebraudLequeux

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Creep in HL

In a creep test, a constant stress σ₀ is applied and the strain γ(t) evolves. The creep compliance J(t) = γ(t)/σ₀ characterizes the material's response.

### Key Physics

**Below yield stress (σ₀ < σ_y)**:
- Bounded creep: γ(t) approaches a finite limit
- Material behaves as a viscoelastic solid
- Stress blocks remain mostly trapped

**Above yield stress (σ₀ > σ_y)**:
- Unbounded creep: γ(t) grows indefinitely
- Material flows like a viscous fluid
- Eventually reaches steady-state flow

### Viscosity Bifurcation

Near the yield stress, the HL model predicts **delayed yielding**:
- For σ₀ slightly above σ_y, the material initially creeps slowly
- After a delay time t_d, flow accelerates dramatically
- This delay diverges as σ₀ → σ_y⁺

The effective viscosity shows bifurcation:
$$
\eta_{eff} = \frac{\sigma_0}{\dot{\gamma}} \to 
\begin{cases}
\infty & \text{if } \sigma_0 < \sigma_y \text{ (bounded creep)} \\
\text{finite} & \text{if } \sigma_0 > \sigma_y \text{ (steady flow)}
\end{cases}
$$

### Creep Compliance

The compliance J(t) = γ(t)/σ₀ depends on the HL parameters:
- α: Controls how easily stress diffuses → affects creep rate
- τ: Sets the characteristic timescale → affects delay time
- σ_c: Yield threshold → determines if creep is bounded

## 3. Load Data

We use polystyrene creep compliance data at different temperatures. The data contains (Time, Creep Compliance) columns.

In [ ]:
# Load polystyrene creep datasets at different temperatures
temperatures = [130, 145, 160, 175, 190]
datasets = {}

for temp in temperatures:
    data_path = os.path.join("..", "data", "creep", "polymers", f"creep_ps{temp}_data.csv")
    # Tab-separated with header "Time\tCreep Compliance"
    raw = np.loadtxt(data_path, delimiter="\t", skiprows=1)
    time_data = raw[:, 0]
    J_t = raw[:, 1]
    datasets[temp] = {"time": time_data, "J_t": J_t}
    print(f"T={temp}°C: {len(time_data)} points, t=[{time_data.min():.3f}, {time_data.max():.1f}] s")

In [ ]:
# Plot all 5 creep curves
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.coolwarm(np.linspace(0.1, 0.9, len(temperatures)))

for i, temp in enumerate(temperatures):
    d = datasets[temp]
    ax.loglog(d["time"], d["J_t"], "o-", color=colors[i], markersize=3, 
              lw=1, label=f"T={temp}°C")

ax.set_xlabel("Time [s]")
ax.set_ylabel("J(t) [1/Pa]")
ax.set_title("Polystyrene Creep Compliance (Temperature Series)")
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

### 4.1 Single Sample (T = 145°C)

We start by fitting the T = 145°C sample as a representative dataset.

In [ ]:
# Fit HL to T=145°C sample
model = HebraudLequeux()

time_145 = datasets[145]["time"]
J_t_145 = datasets[145]["J_t"]

# For creep, we need to specify the applied stress
# We'll use an estimated value based on typical creep experiments
# J = γ/σ, so we can estimate σ from the initial compliance
stress_target = 1000.0  # Pa (typical creep stress for polymer melts)

t0 = time.time()
model.fit(time_145, J_t_145, test_mode="creep", stress_target=stress_target)
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in ["alpha", "tau", "sigma_c"]:
    val = model.parameters.get_value(name)
    print(f"  {name:8s} = {val:.4g}")

phase = model.get_phase_state()
print(f"\nPhase state: {phase}")

In [ ]:
# Plot fit with data
time_fine = np.logspace(
    np.log10(time_145.min()),
    np.log10(time_145.max()),
    200,
)
J_t_pred = model.predict(time_fine)

fig, ax = plt.subplots(figsize=(9, 6))
ax.loglog(time_145, J_t_145, "ko", markersize=5, label="Data T=145°C")
ax.loglog(time_fine, J_t_pred, "-", lw=2, color="C0", label="HL fit")
ax.set_xlabel("Time [s]")
ax.set_ylabel("J(t) [1/Pa]")
ax.set_title(f"HL Creep Fit — α={model.parameters.get_value('alpha'):.3f} ({phase})")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

### 4.2 Temperature Sweep

Now we fit all five temperatures to observe how α evolves with temperature.

In [ ]:
# Fit all 5 samples and track α(T)
fit_results = {}

for temp in temperatures:
    m = HebraudLequeux()
    d = datasets[temp]
    m.fit(d["time"], d["J_t"], test_mode="creep", stress_target=stress_target)
    fit_results[temp] = {
        "alpha": float(m.parameters.get_value("alpha")),
        "tau": float(m.parameters.get_value("tau")),
        "sigma_c": float(m.parameters.get_value("sigma_c")),
        "phase": m.get_phase_state(),
    }

print(f"{'T [°C]':>8s}  {'α':>6s}  {'τ':>10s}  {'σ_c':>10s}  {'Phase'}")
print("-" * 50)
for temp in temperatures:
    r = fit_results[temp]
    print(f"{temp:8d}  {r['alpha']:6.3f}  {r['tau']:10.2e}  {r['sigma_c']:10.2f}  {r['phase']}")

In [ ]:
# Plot α(T) trend
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

temps = list(fit_results.keys())
alpha_vals = [fit_results[t]["alpha"] for t in temps]
tau_vals = [fit_results[t]["tau"] for t in temps]

ax1.plot(temps, alpha_vals, "o-", markersize=10, lw=2, color="C0")
ax1.axhline(0.5, color="red", linestyle="--", alpha=0.7, lw=2, label="Glass transition (α=0.5)")
ax1.set_xlabel("Temperature [°C]")
ax1.set_ylabel("Coupling parameter α")
ax1.set_title("Temperature Effect: α(T)")
ax1.set_ylim(0, 1)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot τ(T) - Arrhenius-like behavior expected
ax2.semilogy(temps, tau_vals, "s-", markersize=10, lw=2, color="C1")
ax2.set_xlabel("Temperature [°C]")
ax2.set_ylabel("Relaxation time τ [s]")
ax2.set_title("Temperature Effect: τ(T)")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Bayesian Inference with NUTS

### 5.1 Run NUTS

In [ ]:
# Bayesian inference with NLSQ warm-start
initial_values = {
    name: model.parameters.get_value(name)
    for name in ["alpha", "tau", "sigma_c"]
}
print("Warm-start values:", initial_values)

# Fast demo config
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

t0 = time.time()
result = model.fit_bayesian(
    time_145,
    J_t_145,
    test_mode="creep",
    stress_target=stress_target,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

### 5.2 Convergence Diagnostics

In [ ]:
# Convergence diagnostics table
diag = result.diagnostics
param_names = ["alpha", "tau", "sigma_c"]

print("Convergence Diagnostics")
print("=" * 50)
print(f"{'Parameter':>10s}  {'R-hat':>8s}  {'ESS':>8s}")
print("-" * 50)
for p in param_names:
    r_hat = diag.get("r_hat", {}).get(p, float("nan"))
    ess = diag.get("ess", {}).get(p, float("nan"))
    print(f"{p:>10s}  {r_hat:8.4f}  {ess:8.0f}")

n_div = diag.get("divergences", diag.get("num_divergences", 0))
print(f"\nDivergences: {n_div}")

In [ ]:
# Trace plots
idata = result.to_inference_data()

axes = az.plot_trace(idata, var_names=param_names, figsize=(12, 6))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Pair plot - look for parameter correlations (identifiability)
axes = az.plot_pair(
    idata,
    var_names=param_names,
    kind="scatter",
    divergences=True,
    figsize=(9, 9),
)
fig = axes.ravel()[0].figure
fig.suptitle("Parameter Correlations (Check Identifiability)", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

### 5.3 Identifiability Analysis

Creep data often has **identifiability challenges** because different parameter combinations can produce similar J(t) curves. Strong correlations in the pair plot indicate parameters that trade off against each other.

In [ ]:
# Compute correlation matrix
posterior = result.posterior_samples

# Stack samples
samples = np.column_stack([posterior[p] for p in param_names])
corr_matrix = np.corrcoef(samples.T)

print("Posterior Correlation Matrix")
print("=" * 50)
print(f"{'':>10s}  {'α':>8s}  {'τ':>8s}  {'σ_c':>8s}")
print("-" * 50)
for i, p1 in enumerate(param_names):
    row = "  ".join([f"{corr_matrix[i, j]:8.3f}" for j in range(len(param_names))])
    print(f"{p1:>10s}  {row}")

print("\nInterpretation:")
print("  |ρ| > 0.7: Strong correlation (potential identifiability issue)")
print("  |ρ| < 0.3: Weak correlation (parameters are identifiable)")

### 5.4 Glass Probability

In [ ]:
# Compute P(glass) = P(α < 0.5)
alpha_samples = np.array(posterior["alpha"])

p_glass = np.mean(alpha_samples < 0.5)
print(f"P(glass) = P(α < 0.5) = {p_glass:.1%}")

# Plot histogram
fig, ax = plt.subplots(figsize=(9, 5))

ax.hist(alpha_samples, bins=50, density=True, alpha=0.7, color="C0", edgecolor="black")
ax.axvline(0.5, color="red", linestyle="--", linewidth=2, label="Glass transition (α=0.5)")

ax.axvspan(0, 0.5, alpha=0.15, color="red", label=f"Glass (P={p_glass:.1%})")
ax.axvspan(0.5, 1.0, alpha=0.15, color="blue", label=f"Fluid (P={1-p_glass:.1%})")

ax.set_xlabel("α (coupling parameter)")
ax.set_ylabel("Posterior density")
ax.set_title(f"Creep-Based Phase Classification: P(glass) = {p_glass:.1%}")
ax.legend()
ax.set_xlim(0, 1)

plt.tight_layout()
display(fig)
plt.close(fig)

### 5.5 Posterior Predictive Check

In [ ]:
# Posterior predictive with 95% credible intervals
n_draws = min(200, len(alpha_samples))
time_pred = np.logspace(
    np.log10(time_145.min()),
    np.log10(time_145.max()),
    100,
)

pred_samples = []
for i in range(n_draws):
    params_i = jnp.array([posterior["alpha"][i], posterior["tau"][i], posterior["sigma_c"][i]])
    pred_i = model.model_function(jnp.array(time_pred), params_i, test_mode="creep")
    pred_samples.append(np.array(pred_i))

pred_samples = np.array(pred_samples)
pred_median = np.median(pred_samples, axis=0)
pred_lo = np.percentile(pred_samples, 2.5, axis=0)
pred_hi = np.percentile(pred_samples, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(9, 6))
ax.fill_between(time_pred, pred_lo, pred_hi, alpha=0.3, color="C0", label="95% CI")
ax.loglog(time_pred, pred_median, "-", lw=2, color="C0", label="Posterior median")
ax.loglog(time_145, J_t_145, "ko", markersize=5, label="Data")
ax.set_xlabel("Time [s]")
ax.set_ylabel("J(t) [1/Pa]")
ax.set_title("Posterior Predictive Check")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

### 5.6 Parameter Summary

In [ ]:
# NLSQ vs Bayesian comparison table
print("Parameter Comparison: NLSQ vs Bayesian")
print("=" * 65)
print(f"{'Param':>10s}  {'NLSQ':>12s}  {'Bayes median':>14s}  {'95% CI':>22s}")
print("-" * 65)

for name in param_names:
    nlsq_val = model.parameters.get_value(name)
    bayes_samples = posterior[name]
    median = float(np.median(bayes_samples))
    lo = float(np.percentile(bayes_samples, 2.5))
    hi = float(np.percentile(bayes_samples, 97.5))
    print(f"{name:>10s}  {nlsq_val:12.4g}  {median:14.4g}  [{lo:.4g}, {hi:.4g}]")

## 6. Save Results

In [ ]:
# Save results
output_dir = os.path.join("..", "outputs", "hl", "creep")
os.makedirs(output_dir, exist_ok=True)

# Save NLSQ point estimates
nlsq_params = {
    name: float(model.parameters.get_value(name))
    for name in ["alpha", "tau", "sigma_c"]
}
with open(os.path.join(output_dir, "nlsq_params_creep.json"), "w") as f:
    json.dump(nlsq_params, f, indent=2)

# Save temperature sweep results
with open(os.path.join(output_dir, "temp_sweep_results.json"), "w") as f:
    json.dump({str(k): v for k, v in fit_results.items()}, f, indent=2)

# Save posterior samples
posterior_dict = {k: np.array(v).tolist() for k, v in posterior.items()}
with open(os.path.join(output_dir, "posterior_creep.json"), "w") as f:
    json.dump(posterior_dict, f)

print(f"Results saved to {output_dir}/")
print(f"  nlsq_params_creep.json: {len(nlsq_params)} parameters")
print(f"  temp_sweep_results.json: {len(fit_results)} temperatures")
print(f"  posterior_creep.json: {len(alpha_samples)} draws")

## Key Takeaways

1. **Creep compliance J(t)** in HL depends on whether the applied stress is above or below the yield stress:
   - Below yield: bounded creep (solid-like)
   - Above yield: unbounded creep with eventual steady flow

2. **Delayed yielding** occurs near the yield stress — the material appears solid initially but eventually flows after a delay time that diverges as σ → σ_y.

3. **Temperature increases fluidness**: Higher T → higher α → more fluid-like behavior, reflecting the thermal activation of stress relaxation.

4. **Identifiability challenges** in creep:
   - Parameters may be correlated (especially τ and σ_c)
   - Wide credible intervals indicate limited data information
   - Consider combining with other protocols (flow curve, relaxation)

5. **Bayesian inference quantifies uncertainty** in phase classification — essential when α is near 0.5.

### Creep-Specific Challenges

| Challenge | Symptom | Solution |
|-----------|---------|----------|
| Parameter correlation | |ρ(τ, σ_c)| > 0.7 | Combine with other protocols |
| Wide CI for σ_c | Uncertainty spans orders of magnitude | More data at different σ₀ |
| Uncertain phase | P(glass) ≈ 50% | Increase samples, combine data |

### Next Steps

- **NB 04**: SAOS (frequency-dependent moduli)
- **NB 05**: Startup (stress overshoot dynamics)
- **NB 06**: LAOS (nonlinear oscillatory response)